# ▒ 로지스틱분석 ▒

## 0. 환경설정

In [28]:
import os
import numpy as np
import pandas as pd
import time  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix   #오차행렬

In [29]:
start= time.time()

sum = 0
for i in range(101):
   sum += i   # su,m = sum + i  

print(sum)
end = time.time()

print(np.round(end-start, 3)) #소수점 3반쩨자리까지 보여줘라


#r : Sys.time

5050
0.001


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 데이터 준비

### 데이터 설명 (Titanic dataset)

- 타이타닉호에 탔던 891명 승객들의 정보
- 승객들의 정보로 사망 여부를 예측하는 모형을 구축하자.
- Survived: 사망 여부 (0: 사망, 1: 생존)
- Pclass: 1=1등석, 2=2등석, 3=3등석
- Sex: male=남성, female=여성
- Age: 나이
- SibSp: 타이타닉 호에 동승한 자매/배우자의 수
- Parch: 타이타닉 호에 동승한 부모/자식의 수
- Ticket: 티켓 번호
- Fare: 승객 요금
- Cabin: 방 호수
- Embarked: 탑승지, C=셰르부르, Q=퀸즈타운, S=사우샘프턴

### 데이터 호출

In [31]:
data_file = "/content/drive/MyDrive/Colab Notebook_2022_2semester/datamining/data/titanic.txt"
titanic = pd.read_csv(data_file)
print(titanic.shape)
titanic.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 성별, 나이, 좌석 클래스로 사망 여부를 예측하자

### 데이터 전처리

- 성별, 나이, 좌석 클래스로 사망 여부를 예측하자. 
- 성별에서 남성은 0, 여성은 1로 변환
- 나이에서 결측치는 평균값으로 대체
- 좌석 클래스에 대한 가변수 생성 (1,2,3등급이므로 2개의 가변수면 충분)

In [32]:
titanic['Sex'] = titanic['Sex'].map({'female':1,'male':0})
titanic['Age'].fillna(value=titanic['Age'].mean(), inplace=True)
titanic['FirstClass'] = titanic['Pclass'].apply(lambda x: 1 if x == 1 else 0)
titanic['SecondClass'] = titanic['Pclass'].apply(lambda x: 1 if x == 2 else 0)

titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FirstClass,SecondClass
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,0,0


- 종속변수와 독립변수 구분하기
- 학습 데이터, 평가 데이터, 시험 데이터 나누기

In [33]:
x_titanic = titanic[['Sex', 'Age', 'FirstClass', 'SecondClass']]
y_titanic = titanic['Survived']

In [34]:
train_x_titanic_f, test_x_titanic, train_y_titanic_f, test_y_titanic = train_test_split(x_titanic, y_titanic, test_size=0.3, random_state=123)
train_x_titanic, valid_x_titanic, train_y_titanic, valid_y_titanic = train_test_split(train_x_titanic_f, train_y_titanic_f, test_size=0.2, random_state=123)

print(train_x_titanic.shape)
print(valid_x_titanic.shape)
print(test_x_titanic.shape)

(498, 4)
(125, 4)
(268, 4)


## 1. 로지스틱 모형 적합하기
### 학습 데이터를 이용해서 로지스틱 모형을 적합하자.

In [35]:
logistic = LogisticRegression(penalty='none')
logistic.fit(train_x_titanic, train_y_titanic)

LogisticRegression(penalty='none')

In [36]:
print(logistic.intercept_)
print(logistic.coef_)

[-1.00527809]
[[ 2.60027081 -0.04209493  2.5517607   1.04817913]]


## 2. 적합된 로지스틱 모형 성능 확인하기
### 예측 정확도 살펴보기 (Accuracy)

In [37]:
print(logistic.score(train_x_titanic, train_y_titanic))
print(logistic.score(test_x_titanic, test_y_titanic))

0.8012048192771084
0.7985074626865671


In [38]:
print(test_x_titanic.head())
print(logistic.predict(test_x_titanic)[0:5])

     Sex        Age  FirstClass  SecondClass
172    1   1.000000           0            0
524    0  29.699118           0            0
452    0  30.000000           1            0
170    0  61.000000           1            0
620    0  27.000000           0            0
[1 0 1 0 0]


### 오차행렬 구하기 (Confusion Matrix)

In [39]:
test_y_prediction = logistic.predict(test_x_titanic)
print(test_y_titanic.sum())
print(test_y_prediction.sum())    #실제로는 98개가 1인데, 내가 예측한 것은 112개가 y=1이라고 함.
confusion_matrix(test_y_titanic, test_y_prediction)

98
112


array([[136,  34],
       [ 20,  78]])

## 3. 라쏘를 활용한 로지스틱 회귀 분석
### 데이터 전처리
- 라쏘를 위해서는 설명 변수를 표준화해야 한다.
- 훈련 자료와 시험 자료를 모두 표준화하자.

In [40]:
from sklearn import preprocessing
from sklearn.svm import l1_min_c

In [41]:
scaler = preprocessing.StandardScaler()
st_train_x_titanic_f = scaler.fit_transform(train_x_titanic_f)
st_train_x_titanic = scaler.transform(train_x_titanic)
st_valid_x_titanic = scaler.transform(valid_x_titanic)
st_test_x_titanic = scaler.transform(test_x_titanic)             #기존에 저장된 train의 평균과 분산으로 표준화시켜줌

### Hyper-parameter tuning
- validation을 이용해 최적의 tuning parameter를 찾는다.

In [42]:

## C scope    #c와 란다는 역수의 관계 : c의 후보군 100개
n_C = 100
Cs = l1_min_c(st_train_x_titanic , train_y_titanic, loss="log") * np.logspace(7, 13, n_C)

intercept_scaling = 1e+4

lm = LogisticRegression(penalty = 'l1', solver='liblinear', \
                        intercept_scaling=intercept_scaling, \
                        warm_start=True)    

In [24]:
#최적의 란다를 찾는 과정

err_vec = []

for C in Cs:
    lm.set_params(C=C)
    lm.fit(st_train_x_titanic, train_y_titanic)
    err_vec.append(1-lm.score(st_valid_x_titanic,valid_y_titanic))  #오분류율을 보고 싶었어 : 


  #c를 100개의 후보군에서 차례대로 뽑을거임. lm을 정의해뒀는데, set_params는 옵션을 바꿔라. 즉, 지금 내가 고려하는 c로 집어넣어라. 


best_C = Cs[list(err_vec==min(err_vec)).index(True)]    # validation에서 오분류율이 작은, 정분류율이 높은 값을 선택하게 됨
print(best_C)
end = time.time()
print("Running time: %.4f"%(end-start))


#C는 란다의 역수 => 즉, 란다 거의 0이라는 거임. 변수가 4개밖에 없어서 전체를 선택했을 가능성이 높아.

76155.61216199491
Running time: 228.7631


### 최적의 tuning parameter를 이용하여 최종 모형 적합하기
- 적합한 모형의 추정 모수와 accuracy 및 오차행렬을 알아보자.

In [27]:
final_lasso = LogisticRegression(penalty = 'l1', C=best_C, solver='liblinear').fit(st_train_x_titanic_f, train_y_titanic_f)  
print(final_lasso.intercept_)
print(final_lasso.coef_)

print(final_lasso.score(st_train_x_titanic_f, train_y_titanic_f))
print(final_lasso.score(st_test_x_titanic, test_y_titanic))

test_y_prediction = final_lasso.predict(st_test_x_titanic)
print(test_y_titanic.sum())
print(test_y_prediction.sum())
confusion_matrix(test_y_titanic, test_y_prediction)

[-0.59146286]
[[ 1.22523454 -0.50894042  1.01145506  0.42607124]]
0.7865168539325843
0.7835820895522388
98
108


array([[136,  34],
       [ 24,  74]])

linear랑 logistic은 다름 => 비교해서 학습하셈~